# 加载并查看数据
## 加载数据

In [43]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import re
import jieba
train = pd.read_csv("./data/train.csv", sep="###__###",header = None,index_col=False)
train.columns = ['ID', 'Age', 'Gender', 'Education', 'Query_List']
print(train.size)
train.head(1)

500000


,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...


In [44]:
# test = pd.read_csv("./data/test.csv", sep="###__###",header = None)
# test.columns = ['ID', 'Query_List']
# test.head(1)

## 构造数组

1

## 思路
###  先走通流程

1. 对分词字典的词频分类
1. 对关键词字典的权重回归分类
1. 用少量数据，看效果

### 尽最大可能使用已有信息
1. 把每次查询的文本对应到可能达到的最细分类，然后合成到粗分类

### 加入人工判断信息
1. 文本结构
1. 情感分析
1. 有效文本长度比重


# BOW预处理和分词

分词工具

https://github.com/fxsjy/jieba

https://github.com/thunlp/THULAC-Python

https://github.com/tsroten/pynlpir

https://github.com/rockyzhengwu/FoolNLTK

https://github.com/baidu/lac

https://github.com/hankcs/HanLP

https://github.com/isnowfy/snownlp

同义词替换

https://github.com/fighting41love/funNLP

https://github.com/huyingxi/Synonyms

近义词聚类




## 去停用词
https://github.com/goto456/stopwords



In [6]:
stopwords = pd.read_csv('./stopwords/cn_stopwords.txt',names=['sw'],index_col=False)

### 自定义停用词

In [7]:
# !cat stopwords/cn_stopwords.txt
# jieba.load_userdict('stopwords/my_stopwords.txt')

## 预处理函数

格式化

In [120]:
import jieba.analyse as analyse

def preprocess(n):
    dimensions = ['Age','Gender','Education']
    t=[]
    for dimension in dimensions:
        d=[[],[]]
        C = list(set(train[dimension]))
        C.pop(0)
        for c in C:
            lines = train[train[dimension]==c]['Query_List'][:n]
            for line in lines:
                try:
                    line=re.sub(r'http\S+\s','http',line)
                    line=re.sub(r' ','空格',line)
                    segs=jieba.lcut(line)
#                     segs = analyse.extract_tags(line)
#                     segs = analyse.textrank(line)
                    segs = list(filter(lambda x:len(x)>1, segs)) #没有解析出来的过滤掉
                    segs = list(filter(lambda x:x not in stopwords, segs)) #把停用词过滤掉
                    d[0].append(" ".join(segs))
                    d[1].append(c)
                except Exception as e:
                    print(e)
                    continue
        t.append(d)
    return t

t=preprocess(300)

## 分类函数

以分词字典的词频为特征，就是BOW模型。

向量为One-hot

In [182]:
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score
import numpy as np

class TextClassifier():

    def __init__(self, classifier=MultinomialNB(),vectorizer = CountVectorizer(ngram_range=(1,4), max_features=20000)):
        self.classifier = classifier
        self.vectorizer = vectorizer

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

    def save_model(self, path):
        dump((self.classifier, self.vectorizer), path)

    def load_model(self, path):
        self.classifier, self.vectorizer = load(path)

    def cv_score(self, X, y, kf=StratifiedKFold(n_splits=5, shuffle=False),score=precision_score, **kw):
        X = self.vectorizer.transform(X)
        y = np.array(y)
        y_pred = y[:]
#         X,y=np.array(X),np.array(y)
        
        for i, j in kf.split(X, y):
            X_train, X_test, y_train = X[i], X[j], y[i]
#             X_train, X_test = X[i], X[j]
#             X_train = [X[t] for t in i]
#             y_train = y[i]
#             X_test = [X[t] for t in j]
#             print(X_train)
#             X_train, X_test, y_train = np.array(X)[i], np.array(X)[j], np.array(y)[i]
            self.fit(X_train,y_train)
            y_pred[j]=self.predict(X_test)
        return score(y, y_pred)

# from sklearn.model_selection import train_test_split
# import random
# random.shuffle(sentences)
# x, y = zip(*s[2])
# x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1234)

### MultinomialNB，CountVectorizer（默认）

In [183]:
c = TextClassifier()
# kf=StratifiedKFold(n_splits=5, shuffle=False)
# for X,y in t:
#         X,y=np.array(X),np.array(y)
#     for i, j in kf.split(x, y):
#         print(np.array(x)[i])
#      kf=StratifiedKFold(n_splits=5, shuffle=False)
c.fit(t[0][0],t[0][1])
print(c.cv_score(t[0][0],t[0][1]))

AttributeError: lower not found

### TfidfVector

In [34]:
text_classifier = TextClassifier(vectorizer=TfidfVectorizer())
text_classifier.fit(x_train, y_train)
print(text_classifier.score(x_test, y_test))

NameError: name 'TextClassifier' is not defined

## SVM

In [ ]:
from sklearn.svm import SVC
text_classifier = TextClassifier(classifier=SVC(), vectorizer=TfidfVectorizer())
text_classifier.fit(x_train, y_train)
print(text_classifier.score(x_test, y_test))

## LDA

In [32]:
import gensim



NameError: name '分类算法' is not defined

## word2vec

## FastText

## textCNN